# Place Cell Analysis

Workflow for analyzing place cells in 2D open-field arena.
Runs the full pipeline and saves results (including summary figures) to a bundle.

---

**Note:** Use **Jupyter Lab** for best experience:

```bash
cd notebook && jupyter lab --no-browser --port=6006
```

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from placecell.dataset import BasePlaceCellDataset
from placecell.visualization import (
    plot_arena_calibration,
    plot_behavior_preview,
    plot_coverage,
    plot_diagnostics,
    plot_footprints,
    plot_occupancy_preview,
    plot_preprocess_steps,
    plot_summary_scatter,
)

## Configuration

In [ ]:
CONFIG_ID = "pcell_config"
DATA_PATH = Path(
    #"/mnt/data/minizero_analysis/202512round/202511_analysis_placecell/"
    "/Volumes/ProcData/minizero_analysis/202512round/202511_analysis_placecell/"
    "20251205/WL25/WL25_20251205.yaml"
)

ds = BasePlaceCellDataset.from_yaml(CONFIG_ID, DATA_PATH)
print(f"Config: {CONFIG_ID}")
print(f"Data: {DATA_PATH}")

## Step 1: Load Data & Deconvolve

In [ ]:
ds.load()

if ds.data_cfg is not None and ds.data_cfg.arena_bounds is not None:
    fig_cal = plot_arena_calibration(
        ds.trajectory,
        ds.data_cfg.arena_bounds,
        arena_size_mm=ds.data_cfg.arena_size_mm,
        mm_per_px=ds.mm_per_px,
        video_frame=ds.behavior_video_frame,
    )
    plt.show()
else:
    print("No arena_bounds configured — skipping area overlay preview")

In [ ]:
ds.preprocess_behavior()

if hasattr(ds, "_preprocess_steps") and ds.data_cfg.arena_size_mm is not None:
    plot_preprocess_steps(ds._preprocess_steps, ds.data_cfg.arena_size_mm)
    plt.show()

In [ ]:
ds.deconvolve(
    progress_bar=lambda x, **kw: tqdm(x, desc="Deconvolving", **kw),
)

## Step 2: Match Events & Compute Occupancy

In [ ]:
ds.match_events()
ds.compute_occupancy()

In [ ]:
plot_behavior_preview(
    ds.trajectory, ds.trajectory_filtered, ds.cfg.behavior.speed_threshold,
    speed_unit="mm/s" if ds.mm_per_px else "px/s",
)
plt.show()

In [ ]:
plot_occupancy_preview(
    ds.trajectory_filtered, ds.occupancy_time,
    ds.valid_mask, ds.x_edges, ds.y_edges,
)
plt.show()

In [ ]:
if ds.max_proj is not None and ds.footprints is not None:
    plot_footprints(ds.max_proj, ds.footprints)
    plt.show()
else:
    print("Max projection or footprints not available.")

## Step 3: Spatial Analysis

In [ ]:
ds.analyze_units(progress_bar=lambda x, **kw: tqdm(x, desc="Analyzing", **kw))

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=ds.spatial.p_value_threshold)
plt.show()

In [ ]:
plot_summary_scatter(
    ds.unit_results,
    p_value_threshold=ds.spatial.p_value_threshold,
    n_shuffles=ds.spatial.n_shuffles,
    min_shift_seconds=ds.spatial.min_shift_seconds,
)
plt.show()

## Place Field Coverage

Place field coverage map showing the fraction of place cells with overlapping fields at each spatial bin.

In [ ]:
place_cell_results = ds.place_cells()
n_place_cells = len(place_cell_results)
p_thresh = ds.spatial.p_value_threshold
n_sig = sum(1 for r in ds.unit_results.values() if r.p_val < p_thresh)
print(f"Significant: {n_sig} / {len(ds.unit_results)}")
print(f"Place cells (sig + stable): {n_place_cells} / {len(ds.unit_results)}")

coverage_map, _, _ = ds.coverage()
plot_coverage(
    coverage_map,
    ds.x_edges, ds.y_edges, ds.valid_mask, n_place_cells,
)
plt.show()

In [ ]:
ds.save_bundle(f"../user_data/bundles/{DATA_PATH.stem}.pcellbundle")